# Procesamiento distribuído con Spark

<img src="./images/apachesparklogo.png" alt="spark" style="width: 300px;"/>


##  Inicializando Spark

Lo primero que debe hacer un programa Spark es crear un objeto `SparkSession`. Se introdujo en Spark 2.0 
y se convirtió en un punto de entrada de cada aplicación Spark.

In [1]:
import findspark
findspark.init()

import pyspark                                 # only run after findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

sc = spark.sparkContext

23/10/18 17:31:16 WARN Utils: Your hostname, danrec-HP-Pavilion-Gaming-Laptop-15-ec0xxx resolves to a loopback address: 127.0.1.1; using 10.8.11.97 instead (on interface wlo1)
23/10/18 17:31:16 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/10/18 17:31:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


El objeto `sparkContext` es la variable de `SparkSession` que se utiliza para operar con RDDs.
Más adelante veremos que el objeto `SQLContext` es la variable  de `SparkSession` que se utiliza para operar con `DataFrames`  y `datasets`.
<img src="./images/sparkSession.PNG" alt="spark" style="width: 300px;"/>

## RDD (Resilient Distributed Dataset)

Los RDDs son estructuras de datos fundamentales de Apache Spark. Un RDD es una colección __inmutable__ de datos distribuidos que pueden ser manipulados en paralelo (en los diferentes nodos del clúster).

* __Resilient__: tolerante a fallos. Poseen autorecuperación en caso de fallos. 
* __Distributed__: los datos en RDD se dividen lógicamente en muchos servidores para que puedan computarse en diferentes nodos del clúster.
* __Dataset__: El conjunto de datos representa registros de los datos con los que trabaja. 

### Características principales

* __In-memory Computation__ (Computación en memoria)

Spark RDD almacena resultados intermedios en memoria distribuida (RAM) en lugar de almacenamiento estable (disco).

* __Lazy Evaluations__ (Evaluación perezosas)

Todas las transformaciones en Apache Spark son perezosas, ya que no calculan sus resultados de inmediato. Spark calcula las transformaciones cuando una acción requiere un resultado.

* __Fault Tolerance__ (Tolerancia a fallos)

Los Spark RDD son tolerantes a fallos ya que rastrean la procedencia de los datos para reconstruir automáticamente la información perdida en caso de fallo. Cada RDD recuerda cómo se creó a partir de otros conjuntos de datos para recrearse a sí mismo.

* __Immutability__ (Inmutabilidad)

Los datos son seguros para compartir entre procesos. Por lo tanto, es una forma de alcanzar la consistencia en los cálculos.

* __Partitioning__ (Particionamiento)

La partición es la unidad fundamental de paralelismo en Spark RDD. Cada partición es una división lógica de datos que es mutable. 

* __Persistence__ (Persistencia)

Los usuarios pueden indicar qué RDD reutilizarán y elegir una estrategia de almacenamiento para ellos (por ejemplo, almacenamiento en memoria o en disco).


###  Creación de RDD's

El usuario puede cargar el conjunto de datos externamente, que puede ser un archivo JSON, un archivo CSV, un archivo de texto o una base de datos a través de JDBC sin una estructura de datos específica.

#### Paralelize
Para crear un objeto RDD podemos usar el método `sc.parallelize()`. Este método toma como argumento una colección u objeto iterable y devuelve un objeto de tipo RDD.

<!--
https://data-flair.training/blogs/spark-rdd-tutorial/

Hay tres formas de crear RDD en Spark, como: datos en almacenamiento estable, otros RDD y paralelizar la recopilación ya existente en el programa del controlador. También se pueden operar Spark RDD en paralelo con una API de bajo nivel que ofrece transformaciones y acciones. Estudiaremos estas operaciones Spark RDD más adelante en esta sección.

Spark RDD también se puede almacenar en caché y particionar manualmente. El almacenamiento en caché es beneficioso cuando usamos RDD varias veces. Y la partición manual es importante para equilibrar correctamente las particiones. Generalmente, las particiones más pequeñas permiten distribuir los datos RDD de manera más equitativa, entre más ejecutores. Por lo tanto, menos particiones facilitan el trabajo.

Los programadores también pueden llamar a un método de persistencia para indicar qué RDD quieren reutilizar en operaciones futuras. Spark mantiene los RDD persistentes en la memoria de forma predeterminada, pero puede volcarlos al disco si no hay suficiente RAM. Los usuarios también pueden solicitar otras estrategias de persistencia, como almacenar el RDD solo en el disco o replicarlo entre máquinas, a través de indicadores para persistir.
-->

In [2]:
# Creamos primero el contexto.  Un objeto  SparkContext representa la conexión 
# con el cluster Spark cluster, y puede ser usado para crear RDDs
sc = spark.sparkContext
array = sc.parallelize([1, 2, 3, 4, 5, 6, 7])

Podemos preguntar por el tipo de datos:

In [3]:
type(array)

pyspark.rdd.RDD

Los elementos de la colección son copiados a un objeto distribuido sobre el cual se puede operar de forma distribuída.  


En realidad, dicho método solo le dice a Spark que distribuya los datos __pero no se realiza hasta que no se ejecute alguna acción__. Entre las acciones posibles tenemos: 
`count`, `collect`, `take`, `reduce`. etc.

Por ejemplo, si queremos sumar todos los elementos del objeto `array`, escribiremos:

In [4]:
array.reduce(lambda a, b: a + b)

28

__Particiones__

Un parámetro importante de la operación `parallelize` es la cantidad de particiones en las que se cortará el conjunto de datos. Normalmente, Spark intenta establecer la cantidad de particiones automáticamente en función del clúster. Sin embargo, es posible configurarlo manualmente. Por ejemplo:

In [5]:
array = sc.parallelize([1, 2, 3, 4, 5, 6, 7], 3)
array.glom().collect()

[[1, 2], [3, 4], [5, 6, 7]]

In [6]:
array.getNumPartitions()

3

#### Datos externos

Los objetos RDD también pueden ser creados a partir de ficheros locales, y de cualquier fuente de almacenamiento compatible (HDFS, HBase, etc.). 

Para convertir un archivo de texto en un RDD, usamos el método `textFile()`.

__Ejemplo 1__ (Fichero CSV)

In [7]:
path = "data/Bus_Breakdown_and_Delays.csv"

# así leeríamos un dataframe        
#datos = spark.read.csv(path,sep=';', header=True)
# pero de momento leemos un RDD 
datos = sc.textFile(path) 
print(datos.count())
type(datos)

28578


pyspark.rdd.RDD

El método `textFile` devuelve un registro por cada línea del archivo.

__Ejemplo 2__ (Fichero txt)

El fichero ['data/shakespeare.txt'](data/shakespeare.txt) contiene las Obras completas de William Shakespeare del [Proyecto Gutenberg](http://www.gutenberg.org/wiki/Página_principal). 

In [8]:
fileName =  'data/shakespeare.txt'

# Creamos un RDD de Spark
shakespeareRDD = sc.textFile(fileName, 8)
print(f"Número de líneas: {shakespeareRDD.count()}")
print (f'Número de particiones: {shakespeareRDD.getNumPartitions()}')

Número de líneas: 122395
Número de particiones: 8


<!--
De forma predeterminada, cada RDD transformado se puede volver a calcular cada vez que ejecuta una acción en él. Sin embargo, también puede conservar un RDD en la memoria usando el método persistente (o caché), en cuyo caso Spark mantendrá los elementos en el clúster para un acceso mucho más rápido la próxima vez que lo consulte. También hay soporte para RDD persistentes en el disco o replicados en múltiples nodos.
-->

## Operaciones con RDD's

Los RDD admiten dos tipos de operaciones:
<img src="./images/operaciones_rdd.PNG" alt="spark" style="width: 300px;"/>


### Transformaciones

Son operaciones que __producen un nuevo RDD__ a partir de los RDD existentes. Toma RDD como entrada y produce uno o más RDD como salida. Los RDD de entrada no se pueden cambiar ya que los RDD son de naturaleza inmutable. Las transformaciones son las que crean el registro de procedencia de los RDD (linaje). Esto es un grafo acíclico dirigido.

Las transformaciones son de naturaleza perezosa, es decir, se ejecutan cuando llamamos a una acción. No se ejecutan inmediatamente. Este diseño permite que Spark funcione de manera más eficiente. 

Por ejemplo, la operación `map` es una transformación que pasa cada elemento del conjunto de datos a través de una función y devuelve un nuevo RDD que representa los resultados.

### Acciones

Los executers ejecutan un cálculo en el conjunto de datos y __devuelven un valor__ (resultado no RDD) al programa controlador. Las acciones ponen en marcha la pereza de RDD.

La operación `reduce` es una acción que agrega todos los elementos del RDD usando alguna función y devuelve el resultado final al programa controlador.

__Ejemplo:__

Queremos contar el número de caracteres del fichero [data/shakespeare.txt](data/shakespeare.txt).

In [9]:
lines = sc.textFile('data/shakespeare.txt')
lineLengths = lines.map(lambda s: len(s))                  # contamos los caracteres de cada línea
totalLength = lineLengths.reduce(lambda a, b: a + b)       # sumamos los caracteres de cada línea

print(f'Numero de caracteres en el fichero:  {totalLength}')

Numero de caracteres en el fichero:  5205583


* La primera línea define un RDD a partir de un fichero. Este conjunto de datos no se carga en la memoria: las líneas son simplemente un puntero al archivo.
* La segunda línea define el objeto `lineLengths` como resultado de una transformación `map`. Nuevamente, `lineLengths` no se calcula de inmediato debido a la pereza.
* Finalmente, ejecutamos la acción `reduce`, que es una acción. En este punto, Spark divide el cálculo en tareas para ejecutarlas en máquinas separadas, y cada máquina ejecuta tanto el `map` como  el `reduce` sobre la partición local, devolviendo solo su respuesta al programa controlador.

In [10]:
type(lines), type(lineLengths), type(totalLength)

(pyspark.rdd.RDD, pyspark.rdd.PipelinedRDD, int)

### Acciones (RDD -> Resultado)

#### Take

Acción encargada de  seleccionar una cantidad `N` de elementos y devolverlos al driver en formato de lista. Debemos asegurarnos de que caben en memoria.

<img src="./images/take.PNG" alt="spark" style="width: 300px;"/>

__Ejemplo:__

In [11]:
# mostrar los 2 primeros elementos
datos = sc.parallelize([1, 2, 3, 4, 5, 6, 7])
datos.take(2)                                       # devuelve al drive 2 elementos

[1, 2]

__Otras variantes__

* `takeSample`:Devuelve una muestra aleatoria

__Ejemplo:__

In [12]:
# Devuleve 3 elementos aleatorios sin reemplazo
datos = sc.parallelize([1, 2, 3, 4, 5, 6, 7])
datos.takeSample(False, 3)

[2, 4, 3]

#### Count

Operación que cuenta el número de elementos en el RDD.

__Ejemplo__

In [13]:
datos = sc.parallelize([10, 20, 30])
datos.count()

3

Tiene otras variantes:

- `countApprox(timeout, confidence=0.95)`: cuenta el número aproximado de valores (más rápido). Cuanto mayor sera `confidence` mayor exactitud y más lento.

- `countApproxDistinct(relativeSD=0.05)`: número aproximado de valores distintos. El valor `relativeSD` indica la precisión: cuanto más pequeño, más exactitud y más lento.

- `countByValue`: cuenta el número de repeticiones de cada elemento. Devuelve un dicionario.

__Ejemplo:__

In [14]:
lista = sc.parallelize([1, 2, 1, 2, 2])
m = lista.countByValue()
m

defaultdict(int, {1: 2, 2: 3})

In [15]:
type(m)

collections.defaultdict

- `countByKey` : Cuenta el número de valores en cada clave, válido para RDDs de la forma (clave,valor)


<img src="./images/countbykey.PNG" alt="spark" style="width: 300px;"/>

__Ejemplo:__

In [16]:
rdd = sc.parallelize([("a", 1), ("b", 1), ("a", 3)])
rdd.countByKey()

defaultdict(int, {'a': 2, 'b': 1})

#### Collect

Esta operación devuelve al driver el RDD al completo como una lista. Equivale a:
```
rdd.take(rdd.count())
```
Debe utilizarse con precaución, solo para RDDs pequeños.

__Ejemplo:__

In [17]:
lista = sc.parallelize([1, 2, 3, 4, 5])
lista.collect()

[1, 2, 3, 4, 5]

#### Reduce

Aggrega todos los elementos de un RDD aplicando un operador binario y conmutativo de forma recurrente a todo el RDD.

Por ejemplo:
```
     reduce( + , [a,b,c,d]) = a + b + c + d 
```     
Hay que tener en cuenta que la operación se puede realizar sobre  una "permutación" de la lista.

<img src="./images/reduce.PNG" alt="spark" style="width: 300px;"/>

__Ejemplo:__

In [18]:
lista = sc.parallelize([1, 2, 3, 4])
lista.reduce(lambda a, b: a + b)

10

__Ejemplo:__

In [19]:
def calcula_minimo(a, b):
    return min(a,b)

lista = sc.parallelize([7, 2, 3, 4])  # la lista no debe ser vacía
lista.reduce(calcula_minimo)

2

También es posible usar las funciones de agregación del módulo  `operator`.  
[https://docs.python.org/3/library/operator.html](https://docs.python.org/3/library/operator.html)

<table class="docutils align-default">
<colgroup>
<col style="width: 26%">
<col style="width: 29%">
<col style="width: 45%">
</colgroup>
<thead>
<tr class="row-odd"><th class="head"><p>Operación</p></th>
<th class="head"><p>Sintaxis Python</p></th>
<th class="head"><p> Funciones en el módulo  operator</p></th>
</tr>
</thead>
<tbody>
<tr class="row-even"><td><p>Addition</p></td>
<td><p><code class="docutils literal notranslate"><span class="pre">a</span> <span class="pre">+</span> <span class="pre">b</span></code></p></td>
<td><p><code class="docutils literal notranslate"><span class="pre">add(a,</span> <span class="pre">b)</span></code></p></td>
</tr>
<tr class="row-odd"><td><p>Concatenation</p></td>
<td><p><code class="docutils literal notranslate"><span class="pre">seq1</span> <span class="pre">+</span> <span class="pre">seq2</span></code></p></td>
<td><p><code class="docutils literal notranslate"><span class="pre">concat(seq1,</span> <span class="pre">seq2)</span></code></p></td>
</tr>
<tr class="row-even"><td><p>Containment Test</p></td>
<td><p><code class="docutils literal notranslate"><span class="pre">obj</span> <span class="pre">in</span> <span class="pre">seq</span></code></p></td>
<td><p><code class="docutils literal notranslate"><span class="pre">contains(seq,</span> <span class="pre">obj)</span></code></p></td>
</tr>
<tr class="row-odd"><td><p>Division</p></td>
<td><p><code class="docutils literal notranslate"><span class="pre">a</span> <span class="pre">/</span> <span class="pre">b</span></code></p></td>
<td><p><code class="docutils literal notranslate"><span class="pre">truediv(a,</span> <span class="pre">b)</span></code></p></td>
</tr>
<tr class="row-even"><td><p>Division</p></td>
<td><p><code class="docutils literal notranslate"><span class="pre">a</span> <span class="pre">//</span> <span class="pre">b</span></code></p></td>
<td><p><code class="docutils literal notranslate"><span class="pre">floordiv(a,</span> <span class="pre">b)</span></code></p></td>
</tr>
<tr class="row-odd"><td><p>Bitwise And</p></td>
<td><p><code class="docutils literal notranslate"><span class="pre">a</span> <span class="pre">&amp;</span> <span class="pre">b</span></code></p></td>
<td><p><code class="docutils literal notranslate"><span class="pre">and_(a,</span> <span class="pre">b)</span></code></p></td>
</tr>
<tr class="row-even"><td><p>Bitwise Exclusive Or</p></td>
<td><p><code class="docutils literal notranslate"><span class="pre">a</span> <span class="pre">^</span> <span class="pre">b</span></code></p></td>
<td><p><code class="docutils literal notranslate"><span class="pre">xor(a,</span> <span class="pre">b)</span></code></p></td>
</tr>
<tr class="row-even"><td><p>Bitwise Or</p></td>
<td><p><code class="docutils literal notranslate"><span class="pre">a</span> <span class="pre">|</span> <span class="pre">b</span></code></p></td>
<td><p><code class="docutils literal notranslate"><span class="pre">or_(a,</span> <span class="pre">b)</span></code></p></td>
</tr>
<tr class="row-odd"><td><p>Exponentiation</p></td>
<td><p><code class="docutils literal notranslate"><span class="pre">a</span> <span class="pre">**</span> <span class="pre">b</span></code></p></td>
<td><p><code class="docutils literal notranslate"><span class="pre">pow(a,</span> <span class="pre">b)</span></code></p></td>
</tr>
<tr class="row-even"><td><p>Identity</p></td>
<td><p><code class="docutils literal notranslate"><span class="pre">a</span> <span class="pre">is</span> <span class="pre">b</span></code></p></td>
<td><p><code class="docutils literal notranslate"><span class="pre">is_(a,</span> <span class="pre">b)</span></code></p></td>
</tr>
<tr class="row-odd"><td><p>Identity</p></td>
<td><p><code class="docutils literal notranslate"><span class="pre">a</span> <span class="pre">is</span> <span class="pre">not</span> <span class="pre">b</span></code></p></td>
<td><p><code class="docutils literal notranslate"><span class="pre">is_not(a,</span> <span class="pre">b)</span></code></p></td>
</tr>
<tr class="row-even"><td><p>Indexed Assignment</p></td>
<td><p><code class="docutils literal notranslate"><span class="pre">obj[k]</span> <span class="pre">=</span> <span class="pre">v</span></code></p></td>
<td><p><code class="docutils literal notranslate"><span class="pre">setitem(obj,</span> <span class="pre">k,</span> <span class="pre">v)</span></code></p></td>
</tr>
<tr class="row-odd"><td><p>Indexed Deletion</p></td>
<td><p><code class="docutils literal notranslate"><span class="pre">del</span> <span class="pre">obj[k]</span></code></p></td>
<td><p><code class="docutils literal notranslate"><span class="pre">delitem(obj,</span> <span class="pre">k)</span></code></p></td>
</tr>
<tr class="row-even"><td><p>Indexing</p></td>
<td><p><code class="docutils literal notranslate"><span class="pre">obj[k]</span></code></p></td>
<td><p><code class="docutils literal notranslate"><span class="pre">getitem(obj,</span> <span class="pre">k)</span></code></p></td>
</tr>
<tr class="row-even"><td><p>Modulo</p></td>
<td><p><code class="docutils literal notranslate"><span class="pre">a</span> <span class="pre">%</span> <span class="pre">b</span></code></p></td>
<td><p><code class="docutils literal notranslate"><span class="pre">mod(a,</span> <span class="pre">b)</span></code></p></td>
</tr>
<tr class="row-odd"><td><p>Multiplication</p></td>
<td><p><code class="docutils literal notranslate"><span class="pre">a</span> <span class="pre">*</span> <span class="pre">b</span></code></p></td>
<td><p><code class="docutils literal notranslate"><span class="pre">mul(a,</span> <span class="pre">b)</span></code></p></td>
</tr>
<tr class="row-even"><td><p>Matrix Multiplication</p></td>
<td><p><code class="docutils literal notranslate"><span class="pre">a</span> <span class="pre">@</span> <span class="pre">b</span></code></p></td>
<td><p><code class="docutils literal notranslate"><span class="pre">matmul(a,</span> <span class="pre">b)</span></code></p></td>
</tr>
<tr class="row-odd"><td><p>Negation (Arithmetic)</p></td>
<td><p><code class="docutils literal notranslate"><span class="pre">-</span> <span class="pre">a</span></code></p></td>
<td><p><code class="docutils literal notranslate"><span class="pre">neg(a)</span></code></p></td>
</tr>
<tr class="row-even"><td><p>Negation (Logical)</p></td>
<td><p><code class="docutils literal notranslate"><span class="pre">not</span> <span class="pre">a</span></code></p></td>
<td><p><code class="docutils literal notranslate"><span class="pre">not_(a)</span></code></p></td>
</tr>
<tr class="row-odd"><td><p>Positive</p></td>
<td><p><code class="docutils literal notranslate"><span class="pre">+</span> <span class="pre">a</span></code></p></td>
<td><p><code class="docutils literal notranslate"><span class="pre">pos(a)</span></code></p></td>
</tr>
<tr class="row-even"><td><p>Right Shift</p></td>
<td><p><code class="docutils literal notranslate"><span class="pre">a</span> <span class="pre">&gt;&gt;</span> <span class="pre">b</span></code></p></td>
<td><p><code class="docutils literal notranslate"><span class="pre">rshift(a,</span> <span class="pre">b)</span></code></p></td>
</tr>
<tr class="row-odd"><td><p>Slice Assignment</p></td>
<td><p><code class="docutils literal notranslate"><span class="pre">seq[i:j]</span> <span class="pre">=</span> <span class="pre">values</span></code></p></td>
<td><p><code class="docutils literal notranslate"><span class="pre">setitem(seq,</span> <span class="pre">slice(i,</span> <span class="pre">j),</span> <span class="pre">values)</span></code></p></td>
</tr>
<tr class="row-even"><td><p>Slice Deletion</p></td>
<td><p><code class="docutils literal notranslate"><span class="pre">del</span> <span class="pre">seq[i:j]</span></code></p></td>
<td><p><code class="docutils literal notranslate"><span class="pre">delitem(seq,</span> <span class="pre">slice(i,</span> <span class="pre">j))</span></code></p></td>
</tr>
<tr class="row-odd"><td><p>Slicing</p></td>
<td><p><code class="docutils literal notranslate"><span class="pre">seq[i:j]</span></code></p></td>
<td><p><code class="docutils literal notranslate"><span class="pre">getitem(seq,</span> <span class="pre">slice(i,</span> <span class="pre">j))</span></code></p></td>
</tr>
<tr class="row-even"><td><p>String Formatting</p></td>
<td><p><code class="docutils literal notranslate"><span class="pre">s</span> <span class="pre">%</span> <span class="pre">obj</span></code></p></td>
<td><p><code class="docutils literal notranslate"><span class="pre">mod(s,</span> <span class="pre">obj)</span></code></p></td>
</tr>
<tr class="row-odd"><td><p>Subtraction</p></td>
<td><p><code class="docutils literal notranslate"><span class="pre">a</span> <span class="pre">-</span> <span class="pre">b</span></code></p></td>
<td><p><code class="docutils literal notranslate"><span class="pre">sub(a,</span> <span class="pre">b)</span></code></p></td>
</tr>
<tr class="row-odd"><td><p>Ordering</p></td>
<td><p><code class="docutils literal notranslate"><span class="pre">a</span> <span class="pre">&lt;</span> <span class="pre">b</span></code></p></td>
<td><p><code class="docutils literal notranslate"><span class="pre">lt(a,</span> <span class="pre">b)</span></code></p></td>
</tr>
<tr class="row-even"><td><p>Ordering</p></td>
<td><p><code class="docutils literal notranslate"><span class="pre">a</span> <span class="pre">&lt;=</span> <span class="pre">b</span></code></p></td>
<td><p><code class="docutils literal notranslate"><span class="pre">le(a,</span> <span class="pre">b)</span></code></p></td>
</tr>
<tr class="row-odd"><td><p>Equality</p></td>
<td><p><code class="docutils literal notranslate"><span class="pre">a</span> <span class="pre">==</span> <span class="pre">b</span></code></p></td>
<td><p><code class="docutils literal notranslate"><span class="pre">eq(a,</span> <span class="pre">b)</span></code></p></td>
</tr>
<tr class="row-even"><td><p>Difference</p></td>
<td><p><code class="docutils literal notranslate"><span class="pre">a</span> <span class="pre">!=</span> <span class="pre">b</span></code></p></td>
<td><p><code class="docutils literal notranslate"><span class="pre">ne(a,</span> <span class="pre">b)</span></code></p></td>
</tr>
<tr class="row-odd"><td><p>Ordering</p></td>
<td><p><code class="docutils literal notranslate"><span class="pre">a</span> <span class="pre">&gt;=</span> <span class="pre">b</span></code></p></td>
<td><p><code class="docutils literal notranslate"><span class="pre">ge(a,</span> <span class="pre">b)</span></code></p></td>
</tr>
<tr class="row-even"><td><p>Ordering</p></td>
<td><p><code class="docutils literal notranslate"><span class="pre">a</span> <span class="pre">&gt;</span> <span class="pre">b</span></code></p></td>
<td><p><code class="docutils literal notranslate"><span class="pre">gt(a,</span> <span class="pre">b)</span></code></p></td>
</tr>
</tbody>
</table>


__Ejemplo:__

In [30]:
import operator as op
lista = sc.parallelize([6, 2, 1])    # la lista no debe ser vacía
lista.reduce(op.add)

9

In [31]:
lista.reduce(op.truediv)

3.0

#### ReduceByKey

Análogo a `reduce`, pero aplicando la función a los elementos que tengan la misma clave
en parejas de la forma (clave, valor).

__Ejemplo__

In [32]:
lista = sc.parallelize([("a", 1), ("b", 1), ("a", 3)])
lista.reduceByKey(op.add).collect()

[('b', 1), ('a', 4)]

### Transformaciones (RDD -> RDD')

Se suele distinguir dos tipos de transformaciones:

* __Transformaciones narrow__: afecta a cada partición por separado. Son `map` y `filter`. Cada partición en el RDD resultado proviene de una única partición del RDD origen.
* __Transformaciones wider__: pueden mezclar los resultados de distintas particiones. Cada partición en el RDD resultado puede provenir de varias particiones distintas del RDD origen. Por ejemplo `groupByKey`.

#### Map

Operación que permite aplicar una función a todos los elementos del RDD. 

__Ejemplo:__

El RDD `shakespeareRDD` consiste en una secuencia de strings (cada una de las líneas del fichero). Sabemos que dentro de cada string tenemos palabras. Convertir el RDD `shakespeareRDD` en un RDD de listas de palabras. 

In [33]:
shakespeareRDD = sc.textFile(fileName)
shakespeareRDD.takeSample(False, 5)

['DROMIO OF EPHESUS. Master, knock the door hard.',
 "  GLOUCESTER. We are the Queen's abjects and must obey.",
 "    That thou might'st think upon these by the seal,",
 '',
 '    No, prelate; such is thy audacious wickedness,']

In [34]:
palabras = shakespeareRDD.map(lambda line: line.strip().split(" "))
palabras.takeSample(False, 2)

[[''],
 ['And', 'just', 'against', 'thy', 'heart', 'make', 'thou', 'a', 'hole,']]

#### flatMap

Esta operación es similar a `map`, pero con la ventaja de que aplana los resultados:
* `map` devuelve lista de listas
* `flatMap` devuelve una única lista  

In [35]:
palabras = shakespeareRDD.flatMap(lambda line: line.strip().split(" "))
palabras.takeSample(False, 10)

['Thus',
 'noble',
 "amaz'd,",
 'Doctor,',
 'for',
 'he',
 'point',
 'yet',
 'well.-',
 'KING']

#### Filter

La operación `filter` devuelve un nuevo RDD que contiene solo los elementos que cumplen una determinada condición. Se considera una transformación de tipo narrow los datos de una partición del RDD resultante pertenecen a una única partición del RDD origen.

__Ejemplo:__

Supongamos que queremos saber cuántas veces se repite la palabra 'what' en el RDD `shakespeareRDD`.

In [36]:
palabras = shakespeareRDD.flatMap(lambda line: line.strip().split(" "))
what_rdd = palabras.filter(lambda x: x == 'we')
what_rdd.count()

2483

#### Group by

Agrupa los datos del RDD mediante una función . Devuelve un RDD creando pares (clave, valor), donde la clave es el resultado de la función, y el valor es una lista de todos los valores del RDD cuyo resultado es dicha clave.

__Ejemplo:__


In [40]:
nombres = sc.parallelize(['Ana', 'Luis', 'Anabel', 'Raúl', 'Lola', 'Ana'])
g = nombres.groupBy(lambda x: x[0])    # agrupamos por la inicial del nombre
g.collect()

[('A', <pyspark.resultiterable.ResultIterable at 0x188ce74ec20>),
 ('L', <pyspark.resultiterable.ResultIterable at 0x188ce761660>),
 ('R', <pyspark.resultiterable.ResultIterable at 0x188ce761600>)]

In [41]:
for (c, v) in g.collect():
    print(c, list(v))

A ['Ana', 'Anabel', 'Ana']
L ['Luis', 'Lola']
R ['Raúl']


## Otras operaciones

    
### Combinación de RDDs

- [join](https://spark.apache.org/docs/3.1.1/api/python/reference/api/pyspark.RDD.join.html): combinando RDDs. También existen leftOuterJoin y rightOuterJoin
        
- [intersection](https://spark.apache.org/docs/3.1.1/api/python/reference/api/pyspark.RDD.intersection.html): intersección de RDDs

- [substract](https://spark.apache.org/docs/3.1.1/api/python/reference/api/pyspark.RDD.subtract.html): elementos de un RDD que no están contenidos en otro

- [union](https://spark.apache.org/docs/3.1.1/api/python/reference/api/pyspark.RDD.union.html): unión de RDDs (en el sentido de juntar valores, no quita repetidos)

- [zip](https://spark.apache.org/docs/3.1.1/api/python/reference/api/pyspark.RDD.zip.html): empareja dos RDDs que se supone que tienen la misma cantidad de elementos
    
### RDDs (clave,valor)

- [contains](https://spark.apache.org/docs/3.1.1/api/python/reference/api/pyspark.SparkConf.contains.html): indica si el RDD contiene una clave

- [lookup](https://spark.apache.org/docs/3.1.1/api/python/reference/api/pyspark.RDD.lookup.html): busca una clave        
- [get](https://spark.apache.org/docs/3.1.1/api/python/reference/api/pyspark.SparkConf.get.html): valor asociado a una clave, o un valor por defecto si no está

- [keyby](https://spark.apache.org/docs/3.1.1/api/python/reference/api/pyspark.RDD.keyBy.html): Crea tuplas

- [keys](https://spark.apache.org/docs/3.1.1/api/python/reference/api/pyspark.RDD.keys.html): las claves de las tuplas (primer elemento)

- [values](https://spark.apache.org/docs/3.1.1/api/python/reference/api/pyspark.RDD.values.html): los valores de las tuplas (segundo elemento)
    
### Operaciones estadísticas:

- [max](https://spark.apache.org/docs/3.1.1/api/python/reference/api/pyspark.RDD.max.html): máximo de la colección

- [mean](https://spark.apache.org/docs/3.1.1/api/python/reference/api/pyspark.RDD.mean.html): la media de la colección

- [meanApprox](https://spark.apache.org/docs/3.1.1/api/python/reference/api/pyspark.RDD.meanApprox.html): la media aproximada de la colección

- [stats](https://spark.apache.org/docs/3.1.1/api/python/reference/api/pyspark.RDD.stats.html?highlight=stats): media, varianza y recuento de elementos en un solo resultado 
        
        
__Ejemplo__:

In [42]:
g.keys().collect()

['A', 'L', 'R']

__Ejemplo:__

In [25]:
nombres = sc.parallelize(['Ana', 'Luis', 'Anabel', 'Raúl', 'Lola'])
salarios = sc.parallelize([100,200,300,400,500])
resultado = nombres.zip(salarios)
resultado.collect()

[('Ana', 100), ('Luis', 200), ('Anabel', 300), ('Raúl', 400), ('Lola', 500)]

In [26]:
# Slario máximo
resultado.values().max()

500

__Ejercicio:__ Calcular los empleados con máximo salario 
<!--    
m = resultado.filter(lambda x : x[1] == 500)
m.collect() 
-->

In [31]:
# Sol: [('Lola', 500)]
rs = resultado.filter(lambda x: x[1] == 500)
rs.collect()

[('Lola', 500)]

## Creación de dataFrames 

Una vez que tenemos un RDD  "limpio" podemos pasarlo a dataframe. Esto se hace en dos pasos:

__1.-__ Convertirlo con `map` en un RDD de Rows <br>
__2.-__ Pasar el RDD de rows a DataFrame <br>

Lo veremos en la siguiente sección.

In [28]:
from pyspark.sql import Row
import datetime

# Paso 1
row_data = resultado.map(lambda tupla: Row(
    nombre=tupla[0], 
    salario=tupla[1],
    fecha=datetime.datetime.now()
    )
)
row_data.take(3)

[Row(nombre='Ana', salario=100, fecha=datetime.datetime(2022, 10, 6, 17, 37, 30, 92906)),
 Row(nombre='Luis', salario=200, fecha=datetime.datetime(2022, 10, 6, 17, 37, 31, 709467)),
 Row(nombre='Anabel', salario=300, fecha=datetime.datetime(2022, 10, 6, 17, 37, 34, 539873))]

In [29]:
# Paso 2
df = spark.createDataFrame(row_data)
df.show()

+------+-------+--------------------+
|nombre|salario|               fecha|
+------+-------+--------------------+
|   Ana|    100|2022-10-06 17:37:...|
|  Luis|    200|2022-10-06 17:37:...|
|Anabel|    300|2022-10-06 17:37:...|
|  Raúl|    400|2022-10-06 17:37:...|
|  Lola|    500|2022-10-06 17:37:...|
+------+-------+--------------------+



## Referencias

* RDD Programming Guide https://spark.apache.org/docs/latest/rdd-programming-guide.html